In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cse493g1/assignments/assignment3/'
FOLDERNAME = 'cse493g1/assignments/assignment3/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the COCO dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/My\ Drive/$FOLDERNAME/cse493g1/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/cse493g1/assignments/assignment3/cse493g1/datasets
/content/drive/My Drive/cse493g1/assignments/assignment3


# Multi-Layer Fully Connected Network Part 2
In this exercise, you will extend your fully connected network from Assignment 2 with Dropout and Normalization Layers. First, you will copy and paste all the necessary parts from Assignment 2. Then you will re-train your model from A2 as a baseline. Next, you will complete the batchnorm and dropout notebook, and then return to this notebook and create an improved model using dropout and normalization.

In [2]:
# Setup cell.
import time
import numpy as np
import matplotlib.pyplot as plt
from cse493g1.classifiers.fc_net import *
from cse493g1.data_utils import get_CIFAR10_data
from cse493g1.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cse493g1.solver import Solver

%matplotlib inline
plt.rcParams["figure.figsize"] = (10.0, 8.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"

%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """Returns relative error."""
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [3]:
# Load the (preprocessed) CIFAR-10 data.
data = get_CIFAR10_data()
for k, v in list(data.items()):
    print(f"{k}: {v.shape}")

X_train: (49000, 3, 32, 32)
y_train: (49000,)
X_val: (1000, 3, 32, 32)
y_val: (1000,)
X_test: (1000, 3, 32, 32)
y_test: (1000,)


# Copy necessary parts from A2.
Fill in the following functions by copying and pasting your answers from A2:
`affine_forward` in `cse493g1/layers.py`
`affine_backward` in `cse493g1/layers.py`
`relu_forward` in `cse493g1/layers.py`
`relu_backward` in `cse493g1/layers.py`
`softmax_loss` in `cse493g1/layers.py`
`sgd_momentum` in `cse493g1/optim.py`
`rmsprop` in `cse493g1/optim.py`
`adam` in `cse493g1/optim.py`



# Train baseline model from A2
Copy and Paste your `FullyConnectedNet` model from `cse493g1/classifiers/fc_net.py` in Assignment 2 into `FullyConnectedNetBasic` in the file `cse493g1/classifiers/fc_net.py` in this assignment. Use the best hyperparms that you found from the previous assignment to train this model. Call this model `best_model_basic`

In [7]:
best_model_basic = None

################################################################################
# TODO: Train the best FullyConnectedNetBasic that you can on CIFAR-10. Store your best model in  #
# the best_model_basic variable.                                                     #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


learning_rates = {1e-3, 2.5e-3, 5e-3}
weight_scales = {5e-2, 1e-2}
num_epoch = {5, 8}
best_val_acc = 0
for lr in learning_rates :
  print("Learning Rate : " + str(lr))
  for ws in weight_scales :
    print("Weight Scales : " + str(ws))
    for ep in num_epoch :
      print("Num Epochs : " +str(ep))
      model = FullyConnectedNetBasic(
          [100, 100, 100, 100, 100],
          weight_scale=ws
      )
      solver = Solver(
          model,
          data,
          num_epochs=ep,
          batch_size=200,
          update_rule='adam',
          optim_config={'learning_rate': lr},
          verbose=False
      )
      solver.train()
      print ("Val Acc : " + str(solver.best_val_acc))
      if solver.best_val_acc > best_val_acc:
        best_model_basic = model
        best_val_acc = solver.best_val_acc

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

Learning Rate : 0.0025
Weight Scales : 0.05
Num Epochs : 8
Val Acc : 0.535
Num Epochs : 5
Val Acc : 0.473
Weight Scales : 0.01
Num Epochs : 8
Val Acc : 0.507
Num Epochs : 5
Val Acc : 0.477
Learning Rate : 0.001
Weight Scales : 0.05
Num Epochs : 8
Val Acc : 0.509
Num Epochs : 5
Val Acc : 0.507
Weight Scales : 0.01
Num Epochs : 8
Val Acc : 0.534
Num Epochs : 5
Val Acc : 0.5
Learning Rate : 0.005
Weight Scales : 0.05
Num Epochs : 8
Val Acc : 0.492
Num Epochs : 5
Val Acc : 0.479
Weight Scales : 0.01
Num Epochs : 8
Val Acc : 0.44
Num Epochs : 5
Val Acc : 0.409


# Evaluate baseline model from A2
Evaluate above baseline model.

In [8]:
y_test_pred = np.argmax(best_model_basic.loss(data['X_test']), axis=1)
y_val_pred = np.argmax(best_model_basic.loss(data['X_val']), axis=1)
print('Validation set accuracy: ', (y_val_pred == data['y_val']).mean())
print('Test set accuracy: ', (y_test_pred == data['y_test']).mean())

Validation set accuracy:  0.535
Test set accuracy:  0.517


# Train improved model
Design a new model in `FullyConnectedNetImproved` in the file `cse493g1/classifiers/fc_net.py`. You can start by having `FullyConnectedNetImproved` be the same design as `FullyConnectedNetBasic`. Next, complete the BatchNormoralization.ipynb and Dropout.ipynb notebooks. Then return to this notebook and complete `FullyConnectedNetImproved` by adding in batchnorm and dropout. Try to beat the accuracy of your baseline model! You may have to adjust your hyperparameters.

In [25]:
best_model_improved = None

################################################################################
# TODO: Train the best FullyConnectedNetImproved that you can on CIFAR-10. You might   #
# find batch/layer normalization and dropout useful. Store your best model in  #
# the best_mode_improved variable.                                                     #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

learning_rates = {3e-3}
weight_scales = {0.01}
num_epoch = {8}
best_val_acc = 0
for lr in learning_rates :
  print("Learning Rate : " + str(lr))
  for ws in weight_scales :
    print("Weight Scales : " + str(ws))
    for ep in num_epoch :
      print("Num Epochs : " +str(ep))
      model = FullyConnectedNetImproved(
          [100, 100],
          weight_scale=ws,
          dropout_keep_ratio=0.88,
          normalization='batchnorm'
      )
      solver = Solver(
          model,
          data,
          num_epochs=ep,
          batch_size=200,
          update_rule='adam',
          optim_config={'learning_rate': lr},
          verbose=False
      )
      solver.train()
      print ("Val Acc : " + str(solver.best_val_acc))
      if solver.best_val_acc > best_val_acc:
        best_model_improved = model
        best_val_acc = solver.best_val_acc

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

Learning Rate : 0.003
Weight Scales : 0.01
Num Epochs : 8
Val Acc : 0.545


# Test Your Model!
Run your best model on the validation and test sets. Are you able to outperform the baseline model that has no Batchnorm or Dropout?

In [26]:
y_test_pred = np.argmax(best_model_improved.loss(data['X_test']), axis=1)
y_val_pred = np.argmax(best_model_improved.loss(data['X_val']), axis=1)
print('Validation set accuracy: ', (y_val_pred == data['y_val']).mean())
print('Test set accuracy: ', (y_test_pred == data['y_test']).mean())

Validation set accuracy:  0.539
Test set accuracy:  0.549
